In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Neighborhoods in Cleveland

In [2]:
# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
CLE_neighborhoods = pd.DataFrame(columns=column_names)

In [3]:
with open('cleveland.geojson') as json_data:
    cleveland_data = json.load(json_data)
CLE_neighborhoods_data = cleveland_data['features']

for data in CLE_neighborhoods_data:
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates'][0][0]
    # neighborhood_latlon = neighborhood_latlon[0]
    neighborhood_latlon = np.mean(neighborhood_latlon, axis=0)
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    CLE_neighborhoods = CLE_neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
CLE_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Buckeye Shaker,41.482072,-81.595033
1,Glenville,41.532918,-81.618742
2,Downtown,41.510783,-81.681775
3,Euclid Green,41.551741,-81.552954
4,Cudell,41.471607,-81.754002


In [4]:
CLE_neighborhoods.shape

(36, 3)

In [5]:
# create map of New York using latitude and longitude values
map_cleveland = folium.Map(location=[41.4993, -81.6944], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(CLE_neighborhoods['Latitude'], CLE_neighborhoods['Longitude'], CLE_neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cleveland)  
    
map_cleveland

## 2. Venues Near the Neighbourhoods

### Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = 'QS44NWUNPK5JSK4G5XJFUGL1VJNCDP2MGBQ5NDOVDPJAANHK' # your Foursquare ID
CLIENT_SECRET = 'XKVRUXX4RTZZKGGWHT03AOHKJ3EDI4XMFH5WZYCNL5FSJOY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Extract Neighbourhood Venues Using Foursquare

In [7]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
cleveland_venues = getNearbyVenues(names=CLE_neighborhoods['Neighborhood'], 
                                   latitudes=CLE_neighborhoods['Latitude'], 
                                   longitudes=CLE_neighborhoods['Longitude'])

Buckeye Shaker
Glenville
Downtown
Euclid Green
Cudell
North Collinwood
Corlett
Central
Clark Fulton
Union Miles Park
Fairfax
South Broadway
Puritas Longmead
Edgewater
Riverside
Forest Hills
North Broadway
Lee Miles
Stockyards
Detroit Shoreway
Woodland Hills
South Collinwood
Old Brooklyn
Jefferson
Tremont
Hough
West Boulevard
Mt Pleasant
Brooklyn Centre
Goodrich-Kirtland Park
Industrial Valley (or Cuyahoga Valley)
Kamm's Corners
Kinsman
Ohio City
Saint Clair-Superior
University


### Number of Venues

In [9]:
print(cleveland_venues.shape)
cleveland_venues

(1064, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Buckeye Shaker,41.482072,-81.595033,Fire Food & Drink,41.483323,-81.589821,American Restaurant
1,Buckeye Shaker,41.482072,-81.595033,North Union Farmers Market at Shaker Square,41.484099,-81.589869,Farmers Market
2,Buckeye Shaker,41.482072,-81.595033,SASA,41.483287,-81.591236,Sushi Restaurant
3,Buckeye Shaker,41.482072,-81.595033,Dewey's Coffee,41.484655,-81.590089,Coffee Shop
4,Buckeye Shaker,41.482072,-81.595033,Shaker Square,41.483876,-81.590217,Plaza
5,Buckeye Shaker,41.482072,-81.595033,Big Al's Diner,41.487763,-81.595583,Diner
6,Buckeye Shaker,41.482072,-81.595033,Batuqui,41.487523,-81.594145,Brazilian Restaurant
7,Buckeye Shaker,41.482072,-81.595033,Loganberry Books,41.487674,-81.591429,Bookstore
8,Buckeye Shaker,41.482072,-81.595033,Cosmic Bobbins,41.483289,-81.589616,Clothing Store
9,Buckeye Shaker,41.482072,-81.595033,Felice Urban Cafe,41.487742,-81.595866,Mediterranean Restaurant


In [10]:
print('There are {} uniques categories.'.format(len(cleveland_venues['Venue Category'].unique())))

There are 224 uniques categories.


## 3. Analyze Each Neighborhood

### Frequency of Venues Grouped by neighborhood

In [11]:
# one hot encoding
cleveland_onehot = pd.get_dummies(cleveland_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cleveland_onehot['Neighborhood'] = cleveland_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cleveland_onehot.columns[-1]] + list(cleveland_onehot.columns[:-1])
cleveland_onehot = cleveland_onehot[fixed_columns]

In [12]:
cleveland_grouped = cleveland_onehot.groupby('Neighborhood').mean().reset_index()
cleveland_grouped

,Neighborhood,ATM,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beer Bar,Big Box Store,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,North Indian Restaurant,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Polish Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tattoo Parlor,Taxi Stand,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Brooklyn Centre,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040816,0.020408,0.000000,0.00,0.000000,0.000000,0.040816,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040816,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.020408,0.000000,0.020408,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061224,0.000000,0.00,0.00,0.000000,0.000000,0.020408,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020408,0.00000

### Top 10 venues for each neighborhood

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cleveland_grouped['Neighborhood']

for ind in np.arange(cleveland_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cleveland_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn Centre,Sandwich Place,Cosmetics Shop,Pharmacy,Intersection,Pizza Place,Big Box Store,Shoe Store,Café,Women's Store,Bar
1,Buckeye Shaker,Discount Store,American Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place,Diner,Clothing Store,Brazilian Restaurant,Chinese Restaurant,Bookstore
2,Central,Grocery Store,Fast Food Restaurant,Auto Garage,College Arts Building,Miscellaneous Shop,Camera Store,Restaurant,Discount Store,Electronics Store,Light Rail Station
3,Clark Fulton,Pharmacy,Pizza Place,Deli / Bodega,Music Venue,Dance Studio,Italian Restaurant,Sandwich Place,Intersection,Lawyer,Spanish Restaurant
4,Corlett,Discount Store,Deli / Bodega,Grocery Store,Bank,Food,Intersection,Convenience Store,Coffee Shop,Restaurant,Snack Place
5,Cudell,Sporting Goods Shop,Sandwich Place,Clothing Store,Department Store,Pharmacy,Gym / Fitness Center,Shopping Mall,Shoe Store,Salon / Barbershop,Pizza Place
6,Detroit Shoreway,Pizza Place,Art Gallery,Bar,Grocery Store,Sandwich Place,Food Truck,Food & Drink Shop,Beach,Theater,Mexican Restaurant
7,Downtown,Gym,Brewery,Theater,Grocery Store,Pedestrian Plaza,Bavarian Restaurant,Sandwich Place,Chinese Restaurant,Performing Arts Venue,Coffee Shop
8,Edgewater,Grocery Store,Diner,Pizza Place,Pharmacy,Pet Store,Gay Bar,Gym / Fitness Center,Café,Fast Food Restaurant,Juice Bar
9,Euclid Green,Sandwich Place,Grocery Store,Auto Workshop,Health Food Store,Fried Chicken Joint,Food,Discount Store,Pizza Place,BBQ Joint,Greek Restaurant


## 4. Cluster Neighborhoods

### K-means Clustering

In [15]:
# set number of clusters
kclusters = 3
cleveland_grouped_clustering = cleveland_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cleveland_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0], dtype=int32)

In [16]:
cleveland_merged = CLE_neighborhoods
# add clustering labels
cleveland_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cleveland_merged = cleveland_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
cleveland_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buckeye Shaker,41.482072,-81.595033,0,Discount Store,American Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place,Diner,Clothing Store,Brazilian Restaurant,Chinese Restaurant,Bookstore
1,Glenville,41.532918,-81.618742,0,Garden,Fast Food Restaurant,Pharmacy,Food Service,Botanical Garden,Park,Sandwich Place,Memorial Site,Food,Bank
2,Downtown,41.510783,-81.681775,0,Gym,Brewery,Theater,Grocery Store,Pedestrian Plaza,Bavarian Restaurant,Sandwich Place,Chinese Restaurant,Performing Arts Venue,Coffee Shop
3,Euclid Green,41.551741,-81.552954,0,Sandwich Place,Grocery Store,Auto Workshop,Health Food Store,Fried Chicken Joint,Food,Discount Store,Pizza Place,BBQ Joint,Greek Restaurant
4,Cudell,41.471607,-81.754002,1,Sporting Goods Shop,Sandwich Place,Clothing Store,Department Store,Pharmacy,Gym / Fitness Center,Shopping Mall,Shoe Store,Salon / Barbershop,Pizza Place


In [17]:
# create map
map_clusters = folium.Map(location=[41.4993, -81.6944], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cleveland_merged['Latitude'], cleveland_merged['Longitude'], cleveland_merged['Neighborhood'], cleveland_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [18]:
C1 = cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 0, cleveland_merged.columns[[0] + list(range(4, cleveland_merged.shape[1]))]]
print('\n','Number of Neighourhoods in Cluster 1 is:',len(cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 0]))
C1


 Number of Neighourhoods in Cluster 1 is: 26


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buckeye Shaker,Discount Store,American Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place,Diner,Clothing Store,Brazilian Restaurant,Chinese Restaurant,Bookstore
1,Glenville,Garden,Fast Food Restaurant,Pharmacy,Food Service,Botanical Garden,Park,Sandwich Place,Memorial Site,Food,Bank
2,Downtown,Gym,Brewery,Theater,Grocery Store,Pedestrian Plaza,Bavarian Restaurant,Sandwich Place,Chinese Restaurant,Performing Arts Venue,Coffee Shop
3,Euclid Green,Sandwich Place,Grocery Store,Auto Workshop,Health Food Store,Fried Chicken Joint,Food,Discount Store,Pizza Place,BBQ Joint,Greek Restaurant
5,North Collinwood,Art Gallery,BBQ Joint,Pizza Place,Food Court,Restaurant,Record Shop,Beach,Sandwich Place,Recreation Center,Pharmacy
6,Corlett,Discount Store,Deli / Bodega,Grocery Store,Bank,Food,Intersection,Convenience Store,Coffee Shop,Restaurant,Snack Place
7,Central,Grocery Store,Fast Food Restaurant,Auto Garage,College Arts Building,Miscellaneous Shop,Camera Store,Restaurant,Discount Store,Electronics Store,Light Rail Station
8,Clark Fulton,Pharmacy,Pizza Place,Deli / Bodega,Music Venue,Dance Studio,Italian Restaurant,Sandwich Place,Intersection,Lawyer,Spanish Restaurant
10,Fairfax,Hotel,Cafeteria,BBQ Joint,Theater,Deli / Bodega,Bank,Bakery,Food & Drink Shop,Restaurant,Mexican Restaurant
12,Puritas Longmead,Pizza Place,Bar,Park,Home Service,Convenience Store,Grocery Store,Dessert Shop,Pharmacy,New American Restaurant,Mexican Restaurant


#### Cluster 2

In [19]:
C2 = cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 1, cleveland_merged.columns[[0] + list(range(4, cleveland_merged.shape[1]))]]
print('\n','Number of Neighourhoods in Cluster 2 is:',len(cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 1]))
C2


 Number of Neighourhoods in Cluster 2 is: 9


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Cudell,Sporting Goods Shop,Sandwich Place,Clothing Store,Department Store,Pharmacy,Gym / Fitness Center,Shopping Mall,Shoe Store,Salon / Barbershop,Pizza Place
9,Union Miles Park,Business Service,Lounge,Food & Drink Shop,Mobile Phone Shop,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dry Cleaner,Donut Shop
11,South Broadway,Sandwich Place,Grocery Store,Discount Store,Pharmacy,Gas Station,Convenience Store,Dive Bar,Diner,Ice Cream Shop,Home Service
14,Riverside,Airport Service,Rental Car Location,Airport Lounge,American Restaurant,Hotel,Boutique,Fast Food Restaurant,Café,Snack Place,Jewelry Store
20,Woodland Hills,Fast Food Restaurant,Discount Store,Chinese Restaurant,Pizza Place,Sandwich Place,Restaurant,Clothing Store,Light Rail Station,Convenience Store,Community Center
21,South Collinwood,Pizza Place,Convenience Store,Food,Discount Store,Business Service,Community Center,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
27,Mt Pleasant,Discount Store,Grocery Store,BBQ Joint,Deli / Bodega,Southern / Soul Food Restaurant,Pharmacy,Gastropub,Gas Station,Electronics Store,Eastern European Restaurant
29,Goodrich-Kirtland Park,Harbor / Marina,Chinese Restaurant,Gas Station,Diner,Fast Food Restaurant,Park,Shoe Store,Eastern European Restaurant,Ethiopian Restaurant,Discount Store
30,Industrial Valley (or Cuyahoga Valley),Cocktail Bar,Auto Workshop,Playground,Miscellaneous Shop,Bus Station,Sandwich Place,Light Rail Station,Bar,College Arts Building,Gaming Cafe


#### Cluster 3 has a small sample size and thus is ignored

In [20]:
print('Number of Neighourhoods in Cluster 3 is:',len(cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 2]))
#print('Number of Neighourhoods in Cluster 4 is:',len(cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 3]))
#print('Number of Neighourhoods in Cluster 5 is:',len(cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 4]))
#print('Number of Neighourhoods in Cluster 6 is:',len(cleveland_merged.loc[cleveland_merged['Cluster Labels'] == 5]))

Number of Neighourhoods in Cluster 3 is: 1


## 6. Analyze Potential Business Opportunities

### Common venues in each cluster

In [21]:
def venue_count_per_cluster(C):
    venueC = []
    for ii in range(0,C.shape[0]):
        for jj in range(1, C.shape[1]):
            venueC.append(C.iloc[ii,jj])
    venueC = pd.DataFrame(venueC)
    return venueC[0].value_counts()

In [22]:
# The most common 10 venues in each cluster
print('Places Cluster 1 has in common (by percentage)')
print(venue_count_per_cluster(C1)[0:10]/C1.shape[0],'\n')
print('Places Cluster 2 has in common (by percentage)')
print(venue_count_per_cluster(C2)[0:10]/C2.shape[0])

Places Cluster 1 has in common (by percentage)
Pizza Place             0.653846
Sandwich Place          0.576923
Grocery Store           0.461538
Bar                     0.423077
Discount Store          0.423077
Pharmacy                0.346154
Fast Food Restaurant    0.269231
Convenience Store       0.269231
Deli / Bodega           0.269231
Bank                    0.230769
Name: 0, dtype: float64 

Places Cluster 2 has in common (by percentage)
Discount Store                 0.555556
Eastern European Restaurant    0.444444
Sandwich Place                 0.444444
Pizza Place                    0.333333
Pharmacy                       0.333333
Electronics Store              0.333333
Convenience Store              0.333333
Ethiopian Restaurant           0.333333
Fast Food Restaurant           0.333333
Gas Station                    0.333333
Name: 0, dtype: float64


#### The most two common two venues in cluster 1 are pizza and sandwich places. However, several neighborhoods in this cluster have neither of them. And this might be a business opportunity.

### Find out the neighbourhoods in Cluster 1 without a Pizza Place and a Sandwich Place

In [23]:
(~C1.isin(['Pizza Place','Sandwich Place'])).all(axis=1)

0     False
1     False
2     False
3     False
5     False
6      True
7      True
8     False
10     True
12    False
13    False
15     True
16    False
17     True
18    False
19    False
22    False
23    False
24    False
25    False
26    False
28    False
31    False
33     True
34    False
35    False
dtype: bool

In [24]:
C1.where((~C1.isin(['Pizza Place','Sandwich Place'])).all(axis=1)).dropna()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Corlett,Discount Store,Deli / Bodega,Grocery Store,Bank,Food,Intersection,Convenience Store,Coffee Shop,Restaurant,Snack Place
7,Central,Grocery Store,Fast Food Restaurant,Auto Garage,College Arts Building,Miscellaneous Shop,Camera Store,Restaurant,Discount Store,Electronics Store,Light Rail Station
10,Fairfax,Hotel,Cafeteria,BBQ Joint,Theater,Deli / Bodega,Bank,Bakery,Food & Drink Shop,Restaurant,Mexican Restaurant
15,Forest Hills,Grocery Store,Convenience Store,Food,Deli / Bodega,Breakfast Spot,Dry Cleaner,Seafood Restaurant,Discount Store,Business Service,Bakery
17,Lee Miles,Fast Food Restaurant,Bar,Wings Joint,Donut Shop,Market,Women's Store,Pharmacy,Shoe Store,Discount Store,Salad Place
33,Ohio City,Brewery,Bar,Art Gallery,American Restaurant,Gastropub,Ice Cream Shop,Market,Café,Gym / Fitness Center,Diner


## Results, Discussion and Conclusion: 
### Pizza and sandwich are the two most popular venues in cluster one neighbourhoods. Six neigborhoods (out of twenty nine) in this cluster have neither of the two. With more careful inspection, we find out that most of these six neighborhoods already have other kinds of food places like bars, restaurants or fast food places. However, Central only has two food places nearby and only one of which is fast food. Therefore, it might be a good idea to open a pizza place or a sandwich place in this neighborhood.